# Importing the MLBox

In [1]:
from mlbox.preprocessing import *
from mlbox.optimisation import *
from mlbox.prediction import *

/home/datageek/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/datageek/anaconda2/lib/python2.7/site-packages/databox/preprocessing/drift/__init__.py:9: UserWarning: ipCluster is starting. Please wait 30 sec and check in terminal that 'the engines appear to have started successfully'. For more information on the ipCluster setting, please read the file 'install.txt' of this package.
  warnings.warn("ipCluster is starting. Please wait 30 sec and check in terminal that 'the engines appear to have started successfully'. For more information on the ipCluster setting, please read the file 'install.txt' of this pac

In [2]:
sep = ','
paths = ["train.csv","test.csv"]
target_name="Survived"

scoring = 'accuracy'
n_folds=3

# reading and cleaning all files 

In [3]:
rd = Reader(sep)
df = rd.train_test_split(paths, target_name)


reading csv : train.csv ...
cleaning data...
CPU time: 50.7687239647 seconds

reading csv : test.csv ...
cleaning data...
CPU time: 0.369373083115 seconds

number of common features :  11

gathering and crunching for train and test datasets
reindexing for train and test datasets
dropping training duplicates
dropping constant variables on training set

number of categorical features :  5
number of numerical features :  6
number of training samples :  891
number of test samples :  418

task :  classification
0.0    549
1.0    342
Name: Survived, dtype: int64
encoding target


In [4]:
dft = Drift_thresholder()
df = dft.fit_transform(df)


computing drifts...
CPU time: 0.936506986618 seconds

Top 10 drifts

('PassengerId', 1.0)
('Name', 0.99704442935827498)
('Ticket', 0.84579063327276627)
('Cabin', 0.59550700418006364)
('Embarked', 0.53834737054254522)
('SibSp', 0.52134352316754273)
('Fare', 0.51744001298932041)
('Sex', 0.51694417436933204)
('Pclass', 0.51382143975575834)
('Parch', 0.50992009325888499)

deleted variables :  ['Name', 'PassengerId', 'Ticket']

dumping drift coefficients into directory : save
drift coefficients dumped


# tuning

In [10]:
opt = Optimiser(scoring, n_folds)

In [11]:
opt.evaluate(None, df)

No parameters set. Defaut configuration is tested

########################################################## testing hyper-parameters... #################################################################

>>> NA ENCODER : {'numerical_strategy': 'mean', 'categorical_strategy': '<NULL>'}

>>> CA ENCODER : {'strategy': 'label_encoding'}

>>> ESTIMATOR : {'sigmoid': 1.0, 'seed': 0, 'max_drop': 50, 'min_split_gain': 0, 'boosting_type': 'gbdt', 'silent': True, 'min_child_samples': 10, 'nthread': -1, 'min_child_weight': 5, 'strategy': 'LightGBM', 'max_bin': 255, 'uniform_drop': False, 'max_depth': -1, 'skip_drop': 0.5, 'num_leaves': 31, 'reg_alpha': 0, 'subsample_for_bin': 50000, 'scale_pos_weight': 1, 'learning_rate': 0.05, 'n_estimators': 500, 'objective': 'binary', 'colsample_bytree': 0.8, 'subsample_freq': 1, 'xgboost_dart_mode': False, 'subsample': 0.9, 'reg_lambda': 0, 'drop_rate': 0.1, 'is_unbalance': False}


MEAN SCORE :  roc_auc  =  0.866703416099
VARIANCE :  0.024505950877  (fold 1

0.86670341609944712

In [7]:
space = {
    
        'ne__numerical_strategy':{"search":"choice",
                                 "space":["mean",0]},
        'ce__strategy':{"search":"choice",
                        "space":["label_encoding","random_projection"]}, 
        'fs__threshold':{"search":"uniform",
                        "space":[0.01,0.3]},    
        'est__colsample_bylevel':{"search":"uniform",
                                  "space":[0.3,0.7]}
    
        }

best = opt.optimise(space, df,3)

/home/datageek/anaconda2/lib/python2.7/site-packages/databox/model/supervised/classification/classifier.py:80: UserWarning: Invalid parameter for classifier LightGBM. Parameter IGNORED. Check the list of available parameters with `classifier.get_params().keys()`
  warnings.warn("Invalid parameter for classifier "+str(self.__strategy)+". Parameter IGNORED. Check the list of available parameters with `classifier.get_params().keys()`")



########################################################## testing hyper-parameters... #################################################################

>>> NA ENCODER : {'numerical_strategy': 0, 'categorical_strategy': '<NULL>'}

>>> CA ENCODER : {'strategy': 'label_encoding'}

>>> FEATURE SELECTOR : {'threshold': 0.0754810040891663, 'strategy': 'l1'}

>>> ESTIMATOR : {'sigmoid': 1.0, 'seed': 0, 'max_drop': 50, 'min_split_gain': 0, 'boosting_type': 'gbdt', 'silent': True, 'min_child_samples': 10, 'nthread': -1, 'min_child_weight': 5, 'strategy': 'LightGBM', 'max_bin': 255, 'uniform_drop': False, 'max_depth': -1, 'skip_drop': 0.5, 'num_leaves': 31, 'reg_alpha': 0, 'subsample_for_bin': 50000, 'scale_pos_weight': 1, 'learning_rate': 0.05, 'n_estimators': 500, 'objective': 'binary', 'colsample_bytree': 0.8, 'subsample_freq': 1, 'xgboost_dart_mode': False, 'subsample': 0.9, 'reg_lambda': 0, 'drop_rate': 0.1, 'is_unbalance': False}


MEAN SCORE :  accuracy  =  0.802469135802
VARIANCE :  0

In [13]:
prd = Predictor()
prd.fit_predict(None,df)


No parameters set. Defaut configuration is tested

fitting the pipeline...
pipeline dumped into directory : save/joblib
CPU time: 1.22438097 seconds

predicting...
CPU time: 0.114058017731 seconds

top 10 predictions :

        0.0       1.0  Survived_predicted
0  0.950160  0.049840                   0
1  0.862099  0.137901                   0
2  0.937391  0.062609                   0
3  0.610432  0.389568                   0
4  0.753652  0.246348                   0
5  0.655973  0.344027                   0
6  0.709572  0.290428                   0
7  0.977180  0.022820                   0
8  0.144067  0.855933                   1
9  0.992767  0.007233                   0

dumping predictions into directory : save


<databox.prediction.predictor.Predictor instance at 0x7f1d7b3f2ea8>